# US Wildfires 1992 - 2015
### An analysis of climate trends and wildfire causes

The following Notebook contains an analysis of 1.88 million US wildfire records spanning 1992-2015. The following are questions and ideas that will be explored:

    - Has wildfire activity seen an increase year over year in the given time span?
    - Has wildfire severity increased year over year?
    - What are the trends of Wildfire causes and how has that changed over time?
    - What is the most frequent reported cause?
    - How does geographic location of wildfire correlate to severity and causes?
    - What is the climatic impact to wildfires over time?
    - Finally what ML models and variables can we employ to help predict causes of wildfires and/or severity of wildfires?

The dataset for wildfires comes in a neatly packaged sqlite database originally sourced from an aggregated dataset provided by data.gov
Daily climate summary data is sourced from the Global Historical Climate Network provided by NOAA and GHCN. This set is a little more raw and will take some significant clean up in order to make it usable and pairable with the wildfire dataset.

In order to join climate data with the wildfire dataset, metadata about weather stations will first need to be analyzed. The Haversine Distance Formula will be used to determine the weather station nearest to each fires lat/long origin.

Both SQL and Python will be used for data exploration as well as my SQL client software of choice, DBeaver.



    

In [1]:
%matplotlib inline
import sqlite3 as sql
import pandas as pd
import numpy as np
import re
import time
import random
import datetime
import matplotlib.pyplot as plt
import statistics
import json
from math import radians, cos, sin, asin, sqrt
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RANSACRegressor, LinearRegression, TheilSenRegressor
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, median_absolute_error, r2_score

from sklearn.svm import SVR
from sklearn.linear_model import Ridge,Lasso,ElasticNet,BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
import seaborn
from IPython.display import Image

![Fires Table Structure](./fires_structure.png)

### Dates are going to be an important factor. Lets Look at sqlite documentation for the data type REALDATE

SQLite does not have a storage class set aside for storing dates and/or times. Instead,the built-in Date And Time Functions of SQLite are capable of storing dates and times as TEXT, REAL, or INTEGER values:

    * TEXT as ISO8601 strings ("YYYY-MM-DD HH:MM:SS.SSS").
    * REAL as Julian day numbers, the number of days since noon in Greenwich on November 24, 4714 B.C. according to the proleptic Gregorian calendar.
    * INTEGER as Unix Time, the number of seconds since 1970-01-01 00:00:00 UTC. 

Applications can chose to store dates and times in any of these formats and freely convert between formats using the built-in date and time functions.

Sounds to me these dates fall under the REAL type with a custom datatype name. So it is Julian Dates.



In [2]:
# Query and load data into Pandas DataFrame
#conn = sql.connect('./FPA_FOD_20170508.sqlite')

#wf_data = pd.read_sql_query(f"SELECT * FROM 'Fires'", conn)

#wf_data.head()

#conn.close()

### Turns out all 1.9 million records are a bit overkill. Lets get this dataset down to a more manageable size. From here there will be a focus on only fires that are Size class C - G. That is any fire which the containment perimeter was 10+ acres.

In [10]:
# get only the columns needed for analysis
conn = sql.connect('./FPA_FOD_20170508.sqlite')

wf_data = pd.read_sql_query(f"""
SELECT OBJECTID, NWCG_REPORTING_UNIT_ID, NWCG_REPORTING_UNIT_NAME, FIRE_NAME,
STATE, LATITUDE, LONGITUDE, FIRE_YEAR, DATE(DISCOVERY_DATE) AS DISCOVERY_DATE, DISCOVERY_TIME, STAT_CAUSE_CODE, STAT_CAUSE_DESCR,
DATE(CONT_DATE) AS CONT_DATE, CONT_TIME, FIRE_SIZE, FIRE_SIZE_CLASS
FROM Fires 
WHERE FIRE_SIZE_CLASS != 'A' AND FIRE_SIZE_CLASS != 'B' AND STATE != 'PR' ORDER BY OBJECTID""", conn)

print(wf_data.head)

<bound method NDFrame.head of         OBJECTID NWCG_REPORTING_UNIT_ID            NWCG_REPORTING_UNIT_NAME  \
0             17                USCAENF            Eldorado National Forest   
1             18                USCAENF            Eldorado National Forest   
2             26                USNMLNF             Lincoln National Forest   
3             38                USNCNCF  National Forests in North Carolina   
4             40                USNCNCF  National Forests in North Carolina   
...          ...                    ...                                 ...   
271416   1880388                USCAMEU                      Mendocino Unit   
271417   1880400                USCARRU                      Riverside Unit   
271418   1880412                USCATUU                         Tulare Unit   
271419   1880420                USCAMVU                    Monte Vista Unit   
271420   1880442                USCALAC  Los Angeles County Fire Department   

             FIRE_NAM

In [17]:
# get truncated version of above query for testing while building formulas and optimization
conn = sql.connect('./FPA_FOD_20170508.sqlite')

wf_data_trunc = pd.read_sql_query(f"""
SELECT OBJECTID, NWCG_REPORTING_UNIT_ID, NWCG_REPORTING_UNIT_NAME, FIRE_NAME,
STATE, LATITUDE, LONGITUDE, FIRE_YEAR, DATE(DISCOVERY_DATE) AS DISCOVERY_DATE, DISCOVERY_TIME, STAT_CAUSE_CODE, STAT_CAUSE_DESCR,
DATE(CONT_DATE) AS CONT_DATE, CONT_TIME, FIRE_SIZE, FIRE_SIZE_CLASS
FROM Fires 
WHERE FIRE_SIZE_CLASS != 'A' AND FIRE_SIZE_CLASS != 'B' AND STATE != 'PR' ORDER BY OBJECTID LIMIT 1000""", conn)

print(wf_data_trunc.head)

<bound method NDFrame.head of      OBJECTID NWCG_REPORTING_UNIT_ID              NWCG_REPORTING_UNIT_NAME  \
0          17                USCAENF              Eldorado National Forest   
1          18                USCAENF              Eldorado National Forest   
2          26                USNMLNF               Lincoln National Forest   
3          38                USNCNCF    National Forests in North Carolina   
4          40                USNCNCF    National Forests in North Carolina   
..        ...                    ...                                   ...   
995      7415                USAROUF              Ouachita National Forest   
996      7417                USGACHF  Chattahoochee-Oconee National Forest   
997      7419                USGACHF  Chattahoochee-Oconee National Forest   
998      7430                USMSMNF       National Forests in Mississippi   
999      7431                USMSMNF       National Forests in Mississippi   

             FIRE_NAME STATE   LA

## Time to Explore and make the station data usable to assign a weather station code to each Wildfire record.

The Stations file is not a delimited file, however each line is a fixed width of characters. This should be turned into a CSV or matrix. From there a table can potentially be created in the sqlite DB so proper relationships can be set. This will also provide a good structure to run the Haversine Distance Equation against. 

In [11]:
def reindex_names_states(lst):
    lst.append(lst[4])
    lst[4] = 'NULL'
    return lst

def read_stations(file):
    stations_list = []
    regex = re.compile("[\',]")
    with open(file, 'r+') as stations:
        for line in stations:
            state = 'NULL' if line[38:40].strip() == '' else line[38:40].strip()
            name = regex.sub(r'', line[41:71].strip())
            gsn = 'NULL' if line[71:75].strip() == '' else line[71:75].strip()
            hcn = 'NULL' if line[75:79].strip() == '' else line[75:79].strip()
            wmo = 'NULL' if line[79:85].strip() == '' else line[79:85].strip()
            station = ','.join(line[:37].split()) + ',' + ','.join([state, name, gsn, hcn, wmo])
            station = station.split(',')
            stations_list.append(station)
    return stations_list

stations = read_stations('ghcnd-stations.txt')



In [5]:
# create a new table and insert the station data

create_stations_sql = """CREATE TABLE IF NOT EXISTS WeatherStations (
                            stationid TEXT PRIMARY KEY,
                            latitude FLOAT64,
                            longitude FLOAT64, 
                            elevation FLOAT64, 
                            state TEXT, 
                            name TEXT,
                            gsn INT16,
                            hcn INT16,
                            wmoid INT32
                            );"""

def create_table(con, create_sql):
    try:
        curs = con.cursor()
        curs.execute(create_sql)
    except Exception as e:
        print(e)
        raise Exception(e)

def insert_stations(con, stations):
    try:
        curs = con.cursor()
        for station in stations:
            insert_sql = f"""INSERT INTO WeatherStations
                            VALUES ('{station[0]}', {float(station[1])}, 
                            {float(station[2])}, {float(station[3])},
                            {station[4] if station[4] == 'NULL' else "'"+station[4]+"'"},
                            '{station[5]}',
                            {0 if station[6] == 'NULL' else 1},
                            {0 if station[7] == 'NULL' else 1},
                            {station[8]}
                            );
                        """
            curs.execute(insert_sql)
    except Exception as e:
        print(e)
        raise Exception(e)

create_table(conn, create_stations_sql)
insert_stations(conn, stations)


UNIQUE constraint failed: WeatherStations.stationid


Exception: UNIQUE constraint failed: WeatherStations.stationid

In [12]:
conn.commit()
conn.close()

## Let's check our newly created table in DBeaver to make sure the creation and inserts are correct.

![stations-loaded](./stations-loaded.png)

### All 118,492 global weather stations have been loaded in the Weather Stations table. Now it's time to use the Haversine Distance Equation to determine which station is closest to each wildfire's origin coordinates.


In [13]:
def hav_dist(lat1, long1, lat2, long2):
    #the radius of planet earth in kilometers
    earth_rad = 6371
    #convert coordinates to radians
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    dlong = long2 - long1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlong / 2)**2
    haversine = 2 * asin(sqrt(a))
    dist_km = earth_rad * haversine
    return dist_km

#### Each daily weather summary file contains a single station's historical data. Each file's name is the station id. Since we are only concerned about weather stations in the US, we only need to load a dataframe with the weather stations that are part of the US Historical Climate Network.

In [14]:
conn = sql.connect('./FPA_FOD_20170508.sqlite')

ws_data = pd.read_sql_query(f"SELECT * FROM 'WeatherStations' WHERE hcn = 1 AND stationid LIKE 'US%' AND state != 'PR'", conn)

conn.close()

ws_data.head()

,stationid,latitude,longitude,elevation,state,name,gsn,hcn,wmoid
0,USC00011084,31.0583,-87.0550,25.9,AL,BREWTON 3 SSE,0,1,NaN
1,USC00012813,30.5467,-87.8808,7.0,AL,FAIRHOPE 2 NE,0,1,NaN
2,USC00013160,32.8347,-88.1342,38.1,AL,GAINESVILLE LOCK,0,1,NaN
3,USC00013511,32.6922,-87.5761,75.9,AL,GREENSBORO,0,1,NaN
4,USC00013816,31.8814,-86.2503,132.0,AL,HIGHLAND HOME,0,1,NaN


### Looking in the ghcnd_hcn directory there are 1219 files. The query above returned 1218 zero based index rows. It appears that all relevant weather stations in the ws_data DF have been loaded.

In [15]:
def nearest_station(lat1, long1, fire_state, fire_id, neighbors=neighbor_states):
    distances = ws_data.apply(
        lambda row: hav_dist(lat1, long1, row['latitude'], row['longitude']), axis=1)
    print(fire_id)
    return [(ws_data.loc[distances.idxmin(), 'stationid']), round(distances[distances.idxmin()], 3)]


### Currently we are working against 272,000 wildfire records and approx 1200 Weather Stations.
Once we are working with the actual historical weather data the size, complexity and amount of memory needed to calculate some of the upcoming tasks is going to increase exponentially. If we had stuck with the full wildfire dataset would have meant to following:

    - The distance between all 1200 weather stations would need to be calculated for each wildfire
    - That is about 2,266,800,000 calculations

Not sure how long that would have taken. Maybe days? Forget about looping through the dataframes with that many calculations.

    - for 272,000 wildfire records and 1200 weather station locations, the number of haversine distance calculations performed is approximately: 326,400,000 

Thats still nothing to balk at. Optimization will have to be considered. Using the pandas apply method with a lambda is significantly more performant than a for loop, but still may not get us the efficiency we actually need.

Lets check the efficiency with the inline timeit function pandas provides against the truncated set of wildfires...

In [18]:
%%timeit
# Need two new columns added to the fire dataframe, one for stationid and another for distance in km between fire and weather station
wf_data_trunc['WEATHER_STATION'], wf_data_trunc['DIST_TO_WS'] = zip(*wf_data_trunc.apply(
    lambda row: nearest_station(row['LATITUDE'], row['LONGITUDE'], row['STATE'], row['OBJECTID']), axis=1))
wf_data_trunc.head()

17
18
26
38
40
41
43
46
71
86
110
112
115
117
118
126
137
179
189
230
269
285
288
300
302
305
306
329
333
353
549
580
583
587
589
590
592
599
609
629
636
658
659
724
733
736
737
745
751
772
774
829
830
831
832
836
859
860
864
865
895
896
899
906
914
951
985
992
1009
1017
1054
1067
1077
1079
1083
1091
1093
1101
1113
1132
1133
1141
1144
1232
1263
1266
1268
1271
1274
1286
1287
1320
1326
1341
1343
1345
1352
1357
1359
1367
1369
1376
1436
1454
1474
1486
1500
1503
1506
1512
1513
1526
1554
1558
1562
1567
1574
1575
1577
1579
1581
1583
1587
1589
1593
1611
1612
1624
1637
1650
1658
1680
1687
1688
1689
1692
1695
1703
1704
1726
1728
1729
1730
1732
1735
1736
1745
1747
1748
1751
1755
1780
1805
1818
1852
1872
1888
1889
1892
1900
1940
1958
1971
1987
1999
2023
2030
2035
2109
2111
2115
2145
2224
2240
2248
2252
2259
2260
2288
2304
2324
2327
2344
2370
2381
2385
2401
2419
2420
2430
2437
2456
2475
2497
2518
2521
2531
2540
2541
2544
2596
2601
2603
2606
2607
2620
2630
2636
2643
2658
2659
2670
2673
2674
2681
269

## 15.8 s ± 933 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

for a set of 1000 wildfires the apply/lambda method takes between 14.8 - 16.8 seconds to complete

    - 1.88m / 1000 = 1889
    - 1889 * 16sec = 30,224 Seconds
    - 30224 seconds = 8.4 Hours

So not quite days for the full fire dataset, but ain't nobody got time for that.

    - 272,000 / 1000 = 272
    - 272 * 16sec = 4352 Seconds
    - 4352 sec = 1.21 Hours

Using the reduced dataset of Size Class C and greater fires, calculating the distance between fire locations and weather locations would take approx. 1.25 - 1.5 hours. Still not ideal, but much more manageable. And since this project is due in a week, optimization is my friend.

Surely, there are things that can be done to optimize this further...

### lets attempt to vectorize the input provided to the Haversine function...

In [19]:
def nearest_station_vector(lat1, long1, fire_state, fire_id, neighbors=neighbor_states):
    distances = ws_data.apply(
        lambda row: hav_dist(lat1, long1, ws_data['latitude'], ws_data['longitude']), axis=1)
    print(fire_id)
    return [(ws_data.loc[distances.idxmin(), 'stationid']), round(distances[distances.idxmin()], 3)]

In [ ]:
%%timeit
# Need two new columns added to the fire dataframe, one for stationid and another for distance in km between fire and weather station
wf_data_trunc['WEATHER_STATION'], wf_data_trunc['DIST_TO_WS'] = zip(*wf_data_trunc.apply(
    lambda row: nearest_station(wf_data_trunc['LATITUDE'], wf_data_trunc['LONGITUDE'], row['STATE'], row['OBJECTID']), axis=1))
wf_data_trunc.head()

In [ ]:
with open('neighbor-states.json') as neighbors:
    neighbor_states = json.load(neighbors)

#condense the array of dicts so its easier/faster to use. The key is now state code and value is list of neighbors. 
neighbor_states = {state_dict['code']: state_dict['Neighborcodes'] for state_dict in neighbor_states}

def nearest_station_opt(lat1, long1, fire_state, fire_id, neighbors=neighbor_states):
    distances = ws_data.apply(
        lambda row: hav_dist(lat1, long1, row['latitude'], row['longitude']) if (row['state'] in neighbors[fire_state]) else None, axis=1)
    print(fire_id)
    return [(ws_data.loc[distances.idxmin(), 'stationid']), round(distances[distances.idxmin()], 3)]


In [16]:
%%timeit

wf_data['WEATHER_STATION'], wf_data['DIST_TO_WS'] = zip(*wf_data.apply(
    lambda row: nearest_station(row['LATITUDE'], row['LONGITUDE'], row['STATE'], row['OBJECTID']), axis=1))
wf_data.head()

1820445
1820448
1820450
1820451
1820452
1820455
1820478
1820479
1820483
1820484
1820488
1820491
1820496
1820499
1820500
1820511
1820515
1820524
1820527
1820528
1820531
1820534
1820535
1820536
1820543
1820544
1820549
1820552
1820558
1820567
1820583
1820587
1820593
1820595
1820603
1820610
1820615
1820618
1820619
1820620
1820628
1820629
1820640
1820651
1820654
1820658
1820659
1820662
1820663
1820664
1820676
1820677
1820682
1820684
1820689
1820710
1820715
1820725
1820737
1820739
1820743
1820744
1820748
1820752
1820754
1820761
1820767
1820775
1820786
1820796
1820799
1820802
1820808
1820809
1820812
1820815
1820820
1820830
1820831
1820832
1820834
1820835
1820836
1820839
1820843
1820846
1820847
1820849
1820851
1820854
1820855
1820858
1820862
1820863
1820900
1820902
1820903
1820907
1820916
1820919
1820929
1820934
1820941
1820955
1820965
1820973
1820981
1820983
1820989
1820992
1820995
1820999
1821004
1821017
1821033
1821040
1821042
1821045
1821052
1821053
1821055
1821057
1821065
1821067
1821068


,OBJECTID,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,FIRE_NAME,STATE,LATITUDE,LONGITUDE,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,WEATHER_STATION,DIST_TO_WS
0,17,USCAENF,Eldorado National Forest,POWER,CA,38.523333,-120.211667,2004,2004-10-06,1415,2.0,Equipment Use,2004-10-21,1000,16823.0,G,USW00023185,113.391
1,18,USCAENF,Eldorado National Forest,FREDS,CA,38.780000,-120.260000,2004,2004-10-13,1618,2.0,Equipment Use,2004-10-17,1800,7700.0,G,USW00023185,88.907
2,26,USNMLNF,Lincoln National Forest,BACHELOR,NM,33.315833,-105.512222,2004,2004-07-20,1405,1.0,Lightning,2004-07-20,1600,10.0,C,USW00023044,185.861
3,38,USNCNCF,National Forests in North Carolina,HOWARD GAP,NC,35.000278,-83.351111,2005,2005-01-27,2200,7.0,Arson,2005-01-28,0300,50.3,C,USC00388887,37.174
4,40,USNCNCF,National Forests in North Carolina,AUSTIN CREEK,NC,36.001667,-81.590000,2005,2005-02-12,1520,5.0,Debris Burning,2005-02-13,0330,125.0,D,USC00441209,123.207


In [18]:
wf_data.loc['WEATHER_STATION']

KeyError: 'WEATHER_STATION'